In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from hc_nlp.pipeline import ThesaurusMatcher, EntityFilter, MapEntityTypes, DateMatcher
from hc_nlp.model_testing import test_ner
from hc_nlp.io import load_text_and_annotations_from_labelstudio
from hc_nlp.spacy_helpers import display_manual_annotations, display_ner_annotations
from hc_nlp import constants

import pprint
pp = pprint.PrettyPrinter(indent=2)

import pandas as pd
import spacy

In [64]:
# LOAD MODELS

# pure NER
nlp_lg = spacy.load("en_core_web_lg")

# custom components
thes = ThesaurusMatcher(nlp_lg, thesaurus_path="../data/labels_all_unambiguous_types_people_orgs.jsonl", case_sensitive=False)
thes_ow = ThesaurusMatcher(nlp_lg, thesaurus_path="../data/labels_all_unambiguous_types_people_orgs.jsonl", 
                                  case_sensitive=False, overwrite_ents=True)
entityfilter = EntityFilter()

# model with people & orgs thesaurus annotations before model
nlp_thes = spacy.load("en_core_web_lg")
nlp_thes.add_pipe(thes, before='ner')
nlp_thes.add_pipe(entityfilter, last=True)

# model with people & orgs thesaurus annotations after model
nlp_thes_aft = spacy.load("en_core_web_lg")
nlp_thes_aft.add_pipe(thes, after='ner')
nlp_thes_aft.add_pipe(entityfilter, last=True)

# model with people & orgs thesaurus annotations after model, overwriting NER annotations
nlp_thes_aft_ow = spacy.load("en_core_web_lg")
nlp_thes_aft_ow.add_pipe(thes_ow, after='ner')
nlp_thes_aft_ow.add_pipe(entityfilter, last=True)

# model with rules for dates
datematcher = DateMatcher(nlp_lg)
nlp_dates_bef = spacy.load("en_core_web_lg")
nlp_dates_bef.add_pipe(datematcher, before='ner')

# model with rules for dates before & thesaurus after
nlp_dates_bef_thes_aft = spacy.load("en_core_web_lg")
nlp_dates_bef_thes_aft.add_pipe(datematcher, before='ner')
nlp_dates_bef_thes_aft.add_pipe(thes, after='ner')
nlp_dates_bef_thes_aft.add_pipe(entityfilter, last=True)

# model with rules for dates before & thesaurus after, with overwrite
nlp_dates_bef_thes_aft_ow = spacy.load("en_core_web_lg")
nlp_dates_bef_thes_aft_ow.add_pipe(datematcher, before='ner')
nlp_dates_bef_thes_aft_ow.add_pipe(thes_ow, after='ner')
nlp_dates_bef_thes_aft_ow.add_pipe(entityfilter, last=True)

# add mapping from Spacy to HC types to all pipelines
mapentitytypes = MapEntityTypes(nlp_lg, validate_mapping=False)
nlp_lg.add_pipe(mapentitytypes)
nlp_thes.add_pipe(mapentitytypes)
nlp_thes_aft.add_pipe(mapentitytypes)
nlp_thes_aft_ow.add_pipe(mapentitytypes)
nlp_dates_bef.add_pipe(mapentitytypes)
nlp_dates_bef_thes_aft.add_pipe(mapentitytypes)
nlp_dates_bef_thes_aft_ow.add_pipe(mapentitytypes)

nlp_lg.pipe_names, nlp_thes.pipe_names, nlp_thes_aft.pipe_names, nlp_thes_aft_ow.pipe_names, nlp_dates_bef.pipe_names, nlp_dates_bef_thes_aft.pipe_names, nlp_dates_bef_thes_aft_ow.pipe_names


2020-12-03 12:48:39,418 - hc_nlp.pipeline - INFO - Loading thesaurus from ../data/labels_all_unambiguous_types_people_orgs.jsonl
2020-12-03 12:48:39,418 - hc_nlp.pipeline - INFO - Loading thesaurus from ../data/labels_all_unambiguous_types_people_orgs.jsonl
2020-12-03 12:48:39,418 - hc_nlp.pipeline - INFO - Loading thesaurus from ../data/labels_all_unambiguous_types_people_orgs.jsonl
2020-12-03 12:48:39,418 - hc_nlp.pipeline - INFO - Loading thesaurus from ../data/labels_all_unambiguous_types_people_orgs.jsonl
2020-12-03 12:48:39,418 - hc_nlp.pipeline - INFO - Loading thesaurus from ../data/labels_all_unambiguous_types_people_orgs.jsonl
2020-12-03 12:48:39,418 - hc_nlp.pipeline - INFO - Loading thesaurus from ../data/labels_all_unambiguous_types_people_orgs.jsonl
2020-12-03 12:48:39,418 - hc_nlp.pipeline - INFO - Loading thesaurus from ../data/labels_all_unambiguous_types_people_orgs.jsonl
2020-12-03 12:48:39,418 - hc_nlp.pipeline - INFO - Loading thesaurus from ../data/labels_all_unam

(['tagger', 'parser', 'ner', 'MapEntityTypes'],
 ['tagger',
  'parser',
  'ThesaurusMatcher',
  'ner',
  'EntityFilter',
  'MapEntityTypes'],
 ['tagger',
  'parser',
  'ner',
  'ThesaurusMatcher',
  'EntityFilter',
  'MapEntityTypes'],
 ['tagger',
  'parser',
  'ner',
  'ThesaurusMatcher',
  'EntityFilter',
  'MapEntityTypes'],
 ['tagger', 'parser', 'DateMatcher', 'ner', 'MapEntityTypes'],
 ['tagger',
  'parser',
  'DateMatcher',
  'ner',
  'ThesaurusMatcher',
  'EntityFilter',
  'MapEntityTypes'],
 ['tagger',
  'parser',
  'DateMatcher',
  'ner',
  'ThesaurusMatcher',
  'EntityFilter',
  'MapEntityTypes'])

In [4]:
data = load_text_and_annotations_from_labelstudio("../labelling/export/2020-11-25-17-28-58.zip", nlp_lg)

len(data)

500

In [23]:
def map_annotations_spacy_to_hc(data):
    new_data = []
    
    for text, annotations in data:
        new_annotations = [(item[0], item[1], constants.SPACY_TO_HC_ENTITY_MAPPING.get(item[2], item[2])) for item in annotations]
        
        new_data.append((text, new_annotations))
        
    return new_data
        
data = map_annotations_spacy_to_hc(data)

In [66]:
def generate_results_table(pipelines: dict, examples: list):
    results = pd.DataFrame()
    for name, p in pipelines.items():
        temp_res = pd.json_normalize(test_ner(p, examples=examples))
        temp_res['pipeline'] = name
        
        results = results.append(temp_res)
    
    results = results.set_index('pipeline')
    
    return results

res = generate_results_table(
    {
        'NER': nlp_lg, 
        'NER_thes_bef': nlp_thes, 
        'NER_thes_aft': nlp_thes_aft, 
        'NER_thes_aft_ow': nlp_thes_aft_ow,
        'NER_datematcher_bef': nlp_dates_bef,
        'NER_datematcher_bef_thes_aft': nlp_dates_bef_thes_aft,
        'NER_datematcher_bef_thes_aft_ow': nlp_dates_bef_thes_aft_ow
#         'NER_datematcher_aft': nlp_dates_aft,
    }, 
    data)

res.T
        

pipeline,NER,NER_thes_bef,NER_thes_aft,NER_thes_aft_ow,NER_datematcher_bef,NER_datematcher_bef_thes_aft,NER_datematcher_bef_thes_aft_ow
ents_p,39.557,42.908,42.8401,43.2594,40.5858,44.3262,44.7478
ents_r,58.9159,56.795,56.4022,56.7164,60.9584,58.9159,59.2302
ents_f,47.3335,48.8844,48.6945,49.0823,48.7284,50.5902,50.9804
support,1274,1274,1274,1274,1274,1274,1274
labels_missing_from_annotations,"[NORP, MONEY, CARDINAL, TIME, PERCENT, FAC, LA...","[NORP, MONEY, CARDINAL, TIME, PERCENT, FAC, LA...","[NORP, MONEY, CARDINAL, TIME, PERCENT, FAC, LA...","[NORP, MONEY, CARDINAL, TIME, PERCENT, FAC, LA...","[NORP, MONEY, CARDINAL, TIME, FAC, LANGUAGE, L...","[NORP, MONEY, CARDINAL, TIME, FAC, LANGUAGE, E...","[NORP, MONEY, CARDINAL, TIME, FAC, LANGUAGE, E..."
ents_per_type.ORG.p,38.9908,37.0763,38.7409,37.797,38.9908,38.7409,37.797
ents_per_type.ORG.r,57.2391,58.9226,53.8721,58.9226,57.2391,53.8721,58.9226
ents_per_type.ORG.f,46.3847,45.5137,45.0704,46.0526,46.3847,45.0704,46.0526
ents_per_type.ORG.support,297,297,297,297,297,297,297
ents_per_type.LOC.p,69.967,73.6842,71.7687,73.6842,69.967,71.7687,73.6842


In [27]:
for text, annotations in data[0:50]:
#     print("NER")
#     display_ner_annotations(text, nlp_lg)
    print("thes people & orgs")
    display_ner_annotations(text, nlp_thes_po)
    print([(ent.text, ent.ent_id_) for ent in nlp_thes_po(text).ents])
    print("thes people & orgs (after)")
    display_ner_annotations(text, nlp_thes_aft_po)
    print([(ent.text, ent.ent_id_) for ent in nlp_thes_aft_po(text).ents])
    print("GT")
    display_manual_annotations(text, annotations)
    print("-----")
    print("-----")

thes people & orgs


[('c.1930-1969', ''), ('Smith', 'https://collection.sciencemuseumgroup.org.uk/people/cp120932'), ('Britain', ''), ('569', '')]
thes people & orgs (after)


[('c.1930-1969', ''), ('Smith Alarm', ''), ('Britain', ''), ('569', '')]
GT


-----
-----
thes people & orgs


[('Italy', '')]
thes people & orgs (after)


...

[('British', ''), ('Wembley', '')]
GT


-----
-----
thes people & orgs


[('Brass X-ray tube', ''), ('Debye-Scherrer', ''), ('A. Taylor', ''), ('Nimonic" Alloys', ''), ('1951', '')]
thes people & orgs (after)


[('Brass X-ray tube', ''), ('Debye-Scherrer', ''), ('A. Taylor', ''), ('Nimonic" Alloys', ''), ('1951', '')]
GT


-----
-----


In [10]:
doc = nlp_lg(data[0][0])

for ent in list(doc.ents):
    print(ent.label_)
    

DATE
ORG
GPE
CARDINAL
